In [20]:
"""
Key Features:
- Iterates through user folders in the raw data path.
- Reads transcript CSV files containing text data.
- Uses Hugging Face's RoBERTa model to extract [CLS] token embeddings for each text segment.
- Handles empty texts, missing files, and batch processing errors gracefully.
- Saves the extracted 768-dimensional RoBERTa features into `.parquet` files.
- Preserves additional timing information (Start_Time, End_Time) if available.
- Feature columns renamed to text_f1, text_f2, ..., text_f768
"""

import os
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaModel, logging
from tqdm import tqdm
import warnings
import gc

logging.set_verbosity_error()
warnings.filterwarnings("ignore")

RAW_DATA_PATH = '../../data/raw'
SAVE_BASE_PATH = '../../data/interim/text_features'
BATCH_SIZE = 8
MAX_LENGTH = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

print("Loading RoBERTa model...")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.to(device)
model.eval()

def get_roberta_embeddings(texts, batch_size=BATCH_SIZE):
    """Extract RoBERTa embeddings from texts with proper memory management"""
    if not texts:
        return np.array([]).reshape(0, 768)
    
    embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
        batch = texts[i:i + batch_size]
        batch = [text if text and text.strip() else "[EMPTY]" for text in batch]
        
        try:
            inputs = tokenizer(
                batch, 
                return_tensors='pt', 
                truncation=True, 
                padding=True, 
                max_length=MAX_LENGTH
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model(**inputs)
                cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
                embeddings.append(cls_embeddings)
            
            del inputs, outputs
            if device.type == 'cuda':
                torch.cuda.empty_cache()
            gc.collect()
        
        except Exception as e:
            print(f"Error processing batch {i//batch_size + 1}: {e}")
            zero_embeddings = np.zeros((len(batch), 768))
            embeddings.append(zero_embeddings)
    
    return np.vstack(embeddings)

def process_user_data(user_path, save_dir, user_name):
    """Process a single user's data"""
    transcript_file = next(
        (f for f in os.listdir(user_path) if f.endswith('_Transcript.csv')), 
        None
    )
    
    if transcript_file is None:
        print(f"⚠️  No transcript file found for {user_name}")
        return False
    
    csv_path = os.path.join(user_path, transcript_file)
    
    try:
        file_size_mb = os.path.getsize(csv_path) / (1024 * 1024)
        print(f"Processing {user_name} ({file_size_mb:.1f}MB)")
        
        df = pd.read_csv(csv_path)
        
        if 'Text' not in df.columns:
            print(f"⚠️  No 'Text' column found for {user_name}")
            return False
        
        texts = df['Text'].fillna('').astype(str).tolist()
        
        if not texts:
            print(f"⚠️  No text data found for {user_name}")
            return False
        
        print(f"Extracting features for {len(texts)} texts...")
        features = get_roberta_embeddings(texts)
        
        if features.size == 0:
            print(f"⚠️  No features extracted for {user_name}")
            return False
        
        # Rename feature columns to text_f1, text_f2, ..., text_f768
        df_features = pd.DataFrame(
            features, 
            columns=[f'text_f{i+1}' for i in range(features.shape[1])]
        )
        
        # Add Start_Time and End_Time if available
        if 'Start_Time' in df.columns:
            df_features['Start_Time'] = df['Start_Time'].reset_index(drop=True)
        else:
            print(f"⚠️  No 'Start_Time' column found for {user_name}")
            
        if 'End_Time' in df.columns:
            df_features['End_Time'] = df['End_Time'].reset_index(drop=True)
        else:
            print(f"⚠️  No 'End_Time' column found for {user_name}")
        
        # Save to parquet
        os.makedirs(save_dir, exist_ok=True)
        output_path = os.path.join(save_dir, 'text_features.parquet')
        df_features.to_parquet(output_path, index=False)
        
        print(f"✅ Saved raw RoBERTa features for {user_name} ({features.shape[0]} samples)")
        return True
        
    except Exception as e:
        print(f"❌ Error processing {user_name}: {e}")
        return False

def main():
    if not os.path.exists(RAW_DATA_PATH):
        print(f"❌ Raw data path does not exist: {RAW_DATA_PATH}")
        return
    
    users = [u for u in os.listdir(RAW_DATA_PATH) 
             if os.path.isdir(os.path.join(RAW_DATA_PATH, u))]
    
    if not users:
        print("❌ No user directories found")
        return
    
    print(f"Found {len(users)} users to process")
    
    successful = 0
    failed = 0
    
    for user in users:
        user_path = os.path.join(RAW_DATA_PATH, user, 'text')
        
        if not os.path.isdir(user_path):
            print(f"⚠️  Text directory not found for {user}")
            failed += 1
            continue
        
        save_dir = os.path.join(SAVE_BASE_PATH, user)
        
        if process_user_data(user_path, save_dir, user):
            successful += 1
        else:
            failed += 1
    
    print(f"\n📊 Processing complete:")
    print(f"✅ Successful: {successful}")
    print(f"❌ Failed: {failed}")
    print(f"📁 Total users: {len(users)}")

if __name__ == "__main__":
    main()


Using device: cpu
Loading RoBERTa model...
Found 3 users to process
Processing 302_P (0.0MB)
Extracting features for 99 texts...


Processing batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.94it/s]


✅ Saved raw RoBERTa features for 302_P (99 samples)
Processing 301_P (0.0MB)
Extracting features for 72 texts...


Processing batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  4.35it/s]

✅ Saved raw RoBERTa features for 301_P (72 samples)
⚠️  No transcript file found for .ipynb_checkpoints

📊 Processing complete:
✅ Successful: 2
❌ Failed: 1
📁 Total users: 3
